###  Imports

In [2]:
import fiftyone.utils.torch
import torch

import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

from dataset import FiftyOneTorchDataset
from model import create_model
from util import add_detections, get_transforms

import config
import pickle

torch.manual_seed(1)

In [3]:
# dataset_name = "coco-2017-validation"
dataset_name = "ImageNet_validation"

# The directory containing the dataset to import
dataset_dir = "C:/Data_drive/Data/Imagenet/ImageNet/imagenet_val_dataset/imagenet_val_dataset"

In [4]:
#fo.core.dataset.delete_non_persistent_datasets()

In [5]:
# Check if the datasets exist on this machine
if fo.core.dataset.dataset_exists(dataset_name):

    fo_dataset = fo.load_dataset(dataset_name) # if the dataset  exists, load it
else:
    if dataset_name == "coco-2017-validation":
        fo_dataset = foz.load_zoo_dataset("coco-2017", "validation")
    else: # if the dataset isnt coco, we will load it from the machine
        fo_dataset = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=fo.types.VOCDetectionDataset,
        name=dataset_name,
        )



if dataset_name == "ImageNet_validation":
    with open('dataset_analysis/imagenet_dict_mapping.pkl', 'rb') as f:
                imagenet_class_mapping = pickle.load(f)
    fo_dataset = fo_dataset.map_labels("ground_truth", imagenet_class_mapping)


#needed to calculate image height and width
fo_dataset.compute_metadata()
#create the session to view the dataset
session = fo.launch_app(fo_dataset)

In [6]:
# create the list of labels needed for evaluation, if evaluating on all labels, leave empty

known_unknowns = ['lizard', 'turtle', 'pen', 'cowboy hat', 'tank']
known_knowns = ['clock', 'vase', 'toaster', 'microwave', 'mouse', 'potted plant', 'sports ball', 'zebra', 'dog', 'bird', 'bench', 'parking meter', 'airplane', 'bicycle']



dataset_class_labels = known_knowns + known_unknowns

model_class_labels = known_knowns + known_unknowns


In [7]:
#get the transformations needed for the images
_, test_transforms = get_transforms()

if len(dataset_class_labels) > 0:

    item_view = fo_dataset.filter_labels("ground_truth",
            F("label").is_in(dataset_class_labels))

    # find the class with the fewest examples
    class_count = item_view.count_values("ground_truth.detections.label")
    smallest_class = min(class_count, key=class_count.get) # find the key of the smallest class

    id = set() # create a set to contain the image ids

    for dataset_class in item_view.distinct("ground_truth.detections.label"): # loop through all of the class labels
        class_view = item_view.filter_labels("ground_truth",
            F("label").is_in(dataset_class)) # create a view from which to sample the class
        sample_ids = class_view.take(class_count[smallest_class], seed = 51) # take the number of classes based on the smallest class

        for sample in sample_ids:
            id.add(sample.id) # add the image ids to the set
    item_view = item_view.select(id) # create a view based on these images

    known_view = item_view.filter_labels("ground_truth",
            F("label").is_in(known_knowns))


    unknown_view = item_view.filter_labels("ground_truth",
            F("label").is_in(known_unknowns))


    # use our dataset and defined transformations
    known_evaluation_dataset = FiftyOneTorchDataset(known_view, test_transforms,
        classes=known_knowns)

    # use our dataset and defined transformations
    unknown_evaluation_dataset = FiftyOneTorchDataset(unknown_view, test_transforms,
        classes=known_unknowns)

    print(f'Evaluating on {len(known_evaluation_dataset)} known samples')
    print(f'Evaluating on {len(unknown_evaluation_dataset)} unknown samples')
else: # if we do not provide labels of interest
    item_view = fo_dataset

    #create an item list for use later
    dataset_class_labels = fo_dataset.distinct("ground_truth.detections.label")


print(f'Evaluating on {len(item_view)} samples')



# use our dataset and defined transformations
evaluation_dataset = FiftyOneTorchDataset(item_view, test_transforms,
        classes=dataset_class_labels)

session.view = item_view

# add a blank line dropped during classification
if model_class_labels[0] != 'background':
     model_class_labels.insert(0,'background')

Evaluating on 742 known samples
Evaluating on 271 unknown samples
Evaluating on 1013 samples


In [8]:
# a helper funciton to evaluate the model on certain datasets
def evaluate_model(model , field_name, evaluation_dataset, fiftyone_Dataset, classes = dataset_class_labels, BCC=False, iou=0.5, eps = 35, labelmap=dataset_class_labels):
    # model - the object detection model
    # field_name - the name used to add the model to FiftyOne
    # evaluation_dataset - the FiftyOne view on which to calculate the data
    # fiftyone_Dataset - the full FiftyOne dataset
    # Classes - the class list on which to evaluate the model
    # BCC - use Bounding box Clustering and Consolidation
    # iou - the iou threshold for nms
    # eps - the eps_neighborhood parameter from DBSCAN
    # labelmap - the mapping of model class number to dataset class number

    add_detections(model, evaluation_dataset, fiftyone_Dataset, field_name=field_name, PRED_CLUSTERING=BCC, labelmap = labelmap, eps = eps)

    evaluation = fo.evaluate_detections(
        item_view,
        field_name,
        classes=classes,
        eval_key=field_name,
        compute_mAP=True,
        iou=iou,
    )

    return evaluation

In [9]:
# create the labelmap for the coco dataset
coco_labels = open("dataset_analysis/coco_labels.txt", "r")
coco_list = coco_labels.read().splitlines() # read each line in as a value in a list
coco_list.insert(0,'background') # add the background class
coco_id = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34,
          35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,
          64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91] # annoyingly, COCO has 90 class ids but only 80 labels
coco = dict(zip(coco_id, coco_list)) # convert it to a dict

## Yolov5

In [10]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

YOLOv5  2022-4-27 torch 1.11.0 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)



Using cache found in C:\Users\blain/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-4-27 torch 1.11.0 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)



Fusing layers... 


Fusing layers... 


YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients


YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients


Adding AutoShape... 


Adding AutoShape... 


In [14]:
iou=.1
pre_trained_known_evaluation = evaluate_model(model,
                                evaluation_dataset = known_evaluation_dataset,
                                field_name = "YOLO_known_predictions",
                                fiftyone_Dataset = fo_dataset,
                                classes = known_knowns,
                                iou=iou,
                                labelmap=coco
                                )

pre_trained_unknown_evaluation = evaluate_model(model,
                                evaluation_dataset = unknown_evaluation_dataset,
                                field_name = "YOLO_unknown_predictions",
                                fiftyone_Dataset = fo_dataset,
                                classes = known_unknowns,
                                iou=iou,
                                labelmap=coco
                                )



print(f'mAP: {pre_trained_known_evaluation.mAP()}')
pre_trained_known_evaluation.print_report()

print(f'mAP: {pre_trained_unknown_evaluation.mAP()}')
pre_trained_unknown_evaluation.print_report()

Using device cuda
 100% |█████████████████| 742/742 [1.7m elapsed, 0s remaining, 7.9 samples/s]      


 100% |█████████████████| 742/742 [1.7m elapsed, 0s remaining, 7.9 samples/s]      


Evaluating detections...


Evaluating detections...


 100% |███████████████| 1013/1013 [7.6s elapsed, 0s remaining, 125.2 samples/s]      


...

Performing IoU sweep...


Performing IoU sweep...


 100% |███████████████| 1013/1013 [4.4s elapsed, 0s remaining, 226.9 samples/s]      


 100% |███████████████| 1013/1013 [4.4s elapsed, 0s remaining, 226.9 samples/s]      


mAP: 0.37250852052044786
               precision    recall  f1-score   support

        clock       0.88      0.33      0.48        63
         vase       0.60      0.67      0.64        70
      toaster       1.00      0.02      0.03        64
    microwave       0.86      0.53      0.65        57
        mouse       0.79      0.57      0.66        65
 potted plant       0.44      0.27      0.34       165
  sports ball       0.93      0.44      0.59        94
        zebra       0.78      0.94      0.86        88
          dog       0.92      0.78      0.85        74
         bird       0.75      0.77      0.76        69
        bench       0.82      0.72      0.77        80
parking meter       0.81      0.74      0.77        76
     airplane       0.93      0.93      0.93        69
      bicycle       0.80      0.80      0.80        79

    micro avg       0.76      0.59      0.67      1113
    macro avg       0.81      0.61      0.65      1113
 weighted avg       0.78      0.59    

In [12]:
iou=.5
pre_trained_known_evaluation = evaluate_model(model,
                                evaluation_dataset = known_evaluation_dataset,
                                field_name = "YOLO_known_predictions",
                                fiftyone_Dataset = fo_dataset,
                                classes = known_knowns,
                                iou=iou,
                                labelmap=coco
                                )

pre_trained_unknown_evaluation = evaluate_model(model,
                                evaluation_dataset = unknown_evaluation_dataset,
                                field_name = "YOLO_unknown_predictions",
                                fiftyone_Dataset = fo_dataset,
                                classes = known_unknowns,
                                iou=iou,
                                labelmap=coco
                                )



print(f'mAP: {pre_trained_known_evaluation.mAP()}')
pre_trained_known_evaluation.print_report()

print(f'mAP: {pre_trained_unknown_evaluation.mAP()}')
pre_trained_unknown_evaluation.print_report()

Using device cuda


C:\Users\blain\Documents\Git\TORCH_CLIP_FRCNN_Trainable\util.py:465: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  boxes.append(torch.tensor([box[:4] for box in np.array(preds[0].cpu())]))


 100% |█████████████████| 742/742 [1.8m elapsed, 0s remaining, 8.7 samples/s]      


 100% |█████████████████| 742/742 [1.8m elapsed, 0s remaining, 8.7 samples/s]      


Evaluating detections...


...

Performing IoU sweep...


Performing IoU sweep...


 100% |███████████████| 1013/1013 [4.3s elapsed, 0s remaining, 232.5 samples/s]      


 100% |███████████████| 1013/1013 [4.3s elapsed, 0s remaining, 232.5 samples/s]      


mAP: 0.37250852052044786
               precision    recall  f1-score   support

        clock       0.71      0.27      0.39        63
         vase       0.58      0.64      0.61        70
      toaster       1.00      0.02      0.03        64
    microwave       0.83      0.51      0.63        57
        mouse       0.74      0.54      0.62        65
 potted plant       0.25      0.16      0.19       165
  sports ball       0.89      0.41      0.57        94
        zebra       0.75      0.90      0.81        88
          dog       0.90      0.77      0.83        74
         bird       0.70      0.72      0.71        69
        bench       0.77      0.69      0.73        80
parking meter       0.61      0.55      0.58        76
     airplane       0.91      0.91      0.91        69
      bicycle       0.72      0.72      0.72        79

    micro avg       0.69      0.53      0.60      1113
    macro avg       0.74      0.56      0.60      1113
 weighted avg       0.70      0.53    

In [13]:

session.view = item_view